# Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q2: April, May, June

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True


C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-c

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyO

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q2 2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-02-quarterly-sales.xlsx'
q2_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q2_2020.drop(q2_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q2_2020['County'] = q2_2020['County'].fillna(method='ffill')
q2_2020['Location Code'] = q2_2020['Location Code'].fillna(method='ffill')
q2_2020['City'] = q2_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q2_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q2_2020['Rounded Up'] = q2_2020['Rounded Up'].map(crosswalk)

q2_2020.head()


C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:312: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,County,Location Code,City,Economic Sector (NAICS Code),2020Q2,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",400000.0,True
1,Beaver County,1002.0,Beaver City,UTILITIES (22),600000.0,True
2,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),29553.0,False
3,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1308568.0,False
4,Beaver County,1002.0,Beaver City,WHOLESALE TRADE-DURABLE GOODS (423),413988.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q2_2020 = pd.pivot_table(q2_2020,values = '2020Q2',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q2_2020 = sectors_q2_2020.fillna(0)


# fix column names
new_names = [replace(item) for item in list(sectors_q2_2020.columns)]
sectors_q2_2020.columns = new_names

# get the total sales tax revenue
sectors_q2_2020['TOTAL'] = sectors_q2_2020[new_names].sum(axis=1)

sectors_q2_2020.head()

,ACCOMMODATION_721,ADMIN__SUPPORT__WASTE_MANAG__REMED_SERVICES_56,AGRICULTURE_FORESTRY_FISHING__HUNTING_11,ARTS_ENTERTAINMENT_AND_RECREATION_71,CONSTRUCTION_23,EDUCATIONAL_SERVICES_61,FINANCE__INSURANCE_52,FOOD_SERVICES__DRINKING_PLACES_722,HEALTH_CARE__SOCIAL_ASSISTANCE_62,INFORMATION_51,MANAGEMENT_OF_COMPANIES__ENTERPRISES_55,MANUFACTURING_31_33,MINING_QUARRYING__OIL__GAS_EXTRACTION_21,OTHER_SERVICES_EXCEPT_PUBLIC_ADMINISTRATION_81,PRIOR_PERIOD_PAYMENTS__REFUNDS,PRIVATE_MOTOR_VEHICLE_SALES,PROFESSIONAL_SCIENTIFIC__TECHNICAL_SERVICES_54,PUBLIC_ADMINISTRATION_92,REAL_ESTATE_RENTAL__LEASING_53,RETAIL_BUILD_MATERIAL_GARDEN_EQUIP__SUPPLIES_DEALERS_444,RETAIL_CLOTHING__CLOTHING_ACCESSORIES_STORES_448,RETAIL_ELECTRONICS__APPLIANCE_STORES_443,RETAIL_FOOD__BEVERAGE_STORES_445,RETAIL_FURNITURE__HOME_FURNISHINGS_STORES_442,RETAIL_GASOLINE_STATIONS_447,RETAIL_GENERAL_MERCHANDISE_STORES_452,RETAIL_HEALTH__PERSONAL_CARE_STORES_446,RETAIL_MISCELLANEOUS_STORE_RETAILERS_453,RETAIL_MOTOR_VEHICLE__PARTS_DEALERS_441,RETAIL_NONSTORE_RETAILERS_454,RETAIL_SPORTING_GOODS_HOBBY_MUSIC__BOOK_STORES_451,SPECIAL_EVENT_SALES,TRANSPORTATION__WAREHOUSING_48_49,UNKNOWN_NONCLASSIFIABLE,UTILITIES_22,WHOLESALE_TRADE_DURABLE_GOODS_423,WHOLESALE_TRADE_ELECTRONIC_MARKETS_425,WHOLESALE_TRADE_NONDURABLE_GOODS_424,TOTAL
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,360525.0,625281.0,1750000.0,1132810.0,10855679.0,69947.0,1510886.0,25740513.0,368700.0,6514315.0,25000.0,3709354.0,60000.0,12382993.0,2622074.0,4500000.0,4496886.0,90000.0,3964900.0,29535937.0,5219505.0,3463441.0,4360567.0,2896641.0,3000000.0,45182823.0,5071921.0,7479307.0,109956893.0,19033396.0,7849449.0,50000.0,498043.0,36566.0,5750000.0,15336406.0,459562.0,11190788.0,357151108.0
Beaver City,988520.0,17144.0,400000.0,7000.0,29553.0,10000.0,457083.0,2263825.0,60000.0,495267.0,1000.0,1308568.0,0.0,1021247.0,142192.0,700000.0,153031.0,0.0,143519.0,1005793.0,102242.0,26765.0,2250000.0,9064.0,3750000.0,554323.0,108995.0,235502.0,828081.0,1871246.0,36255.0,35000.0,20000.0,3747.0,600000.0,413988.0,80000.0,146624.0,20275574.0
Blanding,408578.0,45672.0,1000.0,20000.0,305743.0,50000.0,17919.0,600000.0,60000.0,485187.0,5000.0,77542.0,0.0,363436.0,59705.0,900000.0,143740.0,500000.0,59894.0,2720256.0,80220.0,111496.0,2250000.0,3258.0,1500000.0,748656.0,73134.0,124955.0,394683.0,1473315.0,76601.0,0.0,20000.0,527.0,90000.0,308763.0,8000.0,232607.0,14319887.0
Bountiful,100000.0,975548.0,4000.0,585659.0,1647021.0,81571.0,1116141.0,11839024.0,935892.0,5678745.0,25000.0,2171121.0,5000.0,5958817.0,1671381.0,5500000.0,3301438.0,900000.0,2053241.0,1898525.0,2654922.0,3819206.0,24807045.0,1691416.0,2500000.0,2278386.0,2195173.0,3755609.0,58937846.0,17497067.0,2248783.0,150000.0,35000.0,16822.0,8500000.0,4763409.0,131656.0,641651.0,183072115.0
Brigham,655681.0,186299.0,3000.0,600000.0,1456710.0,28719.0,201424.0,6912003.0,297620.0,2767801.0,15000.0,9242535.0,1000.0,3281464.0,715733.0,3250000.0,796656.0,1000.0,1868664.0,2727400.0,440580.0,1228067.0,11722561.0,507641.0,3407129.0,628164.0,355527.0,896487.0,18008684.0,6288043.0,596162.0,30000.0,45507.0,3732.0,2750000.0,5838242.0,72838.0,243912.0,88071985.0


### Load Q2 2019 sales tax revenue data

In [6]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-02-quarterly-sales.xlsx'
q2_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q2_2019.drop(q2_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q2_2019['County'] = q2_2019['County'].fillna(method='ffill')
q2_2019['Location Code'] = q2_2019['Location Code'].fillna(method='ffill')
q2_2019['City'] = q2_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q2_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q2_2019['Rounded Up'] = q2_2019['Rounded Up'].map(crosswalk)

q2_2019.tail()

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:312: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,County,Location Code,City,Economic Sector (NAICS Code),2019Q2,Rounded Up
2411,Weber County,29040.0,South Ogden,FOOD SERVICES & DRINKING PLACES (722),10298427.0,False
2412,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),1429711.0,False
2413,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,1500000.0,True
2414,Weber County,29040.0,South Ogden,SPECIAL EVENT SALES,45000.0,True
2415,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,165743.0,False


In [7]:
# pivot the table to get sectors as columns with tax values by city
sectors_q2_2019 = pd.pivot_table(q2_2019,values = '2019Q2',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q2_2019 = sectors_q2_2019.fillna(0)

# fix column names
new_names = [replace(item) for item in list(sectors_q2_2019.columns)]
sectors_q2_2019.columns = new_names

# get the total sales tax revenue
sectors_q2_2019['TOTAL'] = sectors_q2_2019[new_names].sum(axis=1)

sectors_q2_2019.head()

,ACCOMMODATION_721,ADMIN__SUPPORT__WASTE_MANAG__REMED_SERVICES_56,AGRICULTURE_FORESTRY_FISHING__HUNTING_11,ARTS_ENTERTAINMENT_AND_RECREATION_71,CONSTRUCTION_23,EDUCATIONAL_SERVICES_61,FINANCE__INSURANCE_52,FOOD_SERVICES__DRINKING_PLACES_722,HEALTH_CARE__SOCIAL_ASSISTANCE_62,INFORMATION_51,MANAGEMENT_OF_COMPANIES__ENTERPRISES_55,MANUFACTURING_31_33,MINING_QUARRYING__OIL__GAS_EXTRACTION_21,OTHER_SERVICES_EXCEPT_PUBLIC_ADMINISTRATION_81,PRIOR_PERIOD_PAYMENTS__REFUNDS,PRIVATE_MOTOR_VEHICLE_SALES,PROFESSIONAL_SCIENTIFIC__TECHNICAL_SERVICES_54,PUBLIC_ADMINISTRATION_92,REAL_ESTATE_RENTAL__LEASING_53,RETAIL_BUILD_MATERIAL_GARDEN_EQUIP__SUPPLIES_DEALERS_444,RETAIL_CLOTHING__CLOTHING_ACCESSORIES_STORES_448,RETAIL_ELECTRONICS__APPLIANCE_STORES_443,RETAIL_FOOD__BEVERAGE_STORES_445,RETAIL_FURNITURE__HOME_FURNISHINGS_STORES_442,RETAIL_GASOLINE_STATIONS_447,RETAIL_GENERAL_MERCHANDISE_STORES_452,RETAIL_HEALTH__PERSONAL_CARE_STORES_446,RETAIL_MISCELLANEOUS_STORE_RETAILERS_453,RETAIL_MOTOR_VEHICLE__PARTS_DEALERS_441,RETAIL_NONSTORE_RETAILERS_454,RETAIL_SPORTING_GOODS_HOBBY_MUSIC__BOOK_STORES_451,SPECIAL_EVENT_SALES,TRANSPORTATION__WAREHOUSING_48_49,UNKNOWN_NONCLASSIFIABLE,UTILITIES_22,WHOLESALE_TRADE_DURABLE_GOODS_423,WHOLESALE_TRADE_ELECTRONIC_MARKETS_425,WHOLESALE_TRADE_NONDURABLE_GOODS_424,TOTAL
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,700000.0,566647.0,1250000.0,1600920.0,8794650.0,99011.0,1790289.0,24651818.0,498511.0,9244154.0,20000.0,3599673.0,25000.0,11024408.0,3859810.0,3000000.0,2182874.0,35000.0,3830247.0,23085674.0,7224842.0,7042705.0,4026871.0,3546430.0,2887348.0,41378967.0,5619710.0,6320643.0,89772175.0,7550237.0,7817656.0,20000.0,600000.0,0.0,8250000.0,11443316.0,25000.0,7358148.0,310742734.0
Beaver City,1423694.0,31711.0,250000.0,7000.0,35000.0,15000.0,220378.0,2737418.0,50000.0,722352.0,1000.0,698857.0,0.0,889776.0,719732.0,500000.0,96364.0,0.0,100714.0,564664.0,58911.0,23509.0,1750000.0,4000.0,3500000.0,444185.0,72269.0,143038.0,615787.0,587176.0,23446.0,20000.0,4000.0,0.0,600000.0,524356.0,25000.0,203124.0,17662461.0
Blanding,1386710.0,48945.0,1000.0,90000.0,210655.0,250000.0,25496.0,700000.0,60000.0,483587.0,5000.0,164496.0,0.0,260314.0,189826.0,800000.0,61873.0,500000.0,93226.0,1948647.0,62024.0,39162.0,2000000.0,7000.0,1250000.0,634468.0,58423.0,125019.0,305191.0,656070.0,43412.0,0.0,25000.0,0.0,100000.0,598802.0,1000.0,402846.0,13588192.0
Bountiful,150000.0,2460364.0,2000.0,1105872.0,2074898.0,104656.0,1151100.0,13673812.0,987748.0,5176705.0,20000.0,2685170.0,10000.0,6990239.0,4448948.0,5250000.0,2157862.0,600000.0,2670871.0,1318455.0,2791903.0,3289392.0,20779545.0,1808556.0,2500000.0,1705541.0,1947420.0,3157634.0,53853811.0,7669928.0,1181965.0,90000.0,45000.0,0.0,7000000.0,3651271.0,9000.0,715392.0,165235058.0
Brigham,2250000.0,188781.0,3000.0,800000.0,1527976.0,58893.0,292577.0,7047410.0,499834.0,2584387.0,25000.0,6400432.0,1000.0,3013123.0,678568.0,2500000.0,391323.0,1000.0,1731936.0,1916996.0,392600.0,1059660.0,9955541.0,529234.0,3500000.0,1045589.0,485876.0,1491898.0,15865705.0,3295918.0,534664.0,90000.0,70000.0,0.0,2500000.0,5046056.0,10000.0,320490.0,78105467.0


### Load municipalities and townships shapefile and join to formatted sales tax data

In [8]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [9]:
# merge the 2020 sdf with the pivoted data
cities_q2_2020 = cities_sdf.merge(sectors_q2_2020, left_on= "NAME", right_on="City", how="inner")
cities_q2_2020.head(10)

,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,ACCOMMODATION_721,ADMIN__SUPPORT__WASTE_MANAG__REMED_SERVICES_56,AGRICULTURE_FORESTRY_FISHING__HUNTING_11,ARTS_ENTERTAINMENT_AND_RECREATION_71,CONSTRUCTION_23,EDUCATIONAL_SERVICES_61,FINANCE__INSURANCE_52,FOOD_SERVICES__DRINKING_PLACES_722,HEALTH_CARE__SOCIAL_ASSISTANCE_62,INFORMATION_51,MANAGEMENT_OF_COMPANIES__ENTERPRISES_55,MANUFACTURING_31_33,MINING_QUARRYING__OIL__GAS_EXTRACTION_21,OTHER_SERVICES_EXCEPT_PUBLIC_ADMINISTRATION_81,PRIOR_PERIOD_PAYMENTS__REFUNDS,PRIVATE_MOTOR_VEHICLE_SALES,PROFESSIONAL_SCIENTIFIC__TECHNICAL_SERVICES_54,PUBLIC_ADMINISTRATION_92,REAL_ESTATE_RENTAL__LEASING_53,RETAIL_BUILD_MATERIAL_GARDEN_EQUIP__SUPPLIES_DEALERS_444,RETAIL_CLOTHING__CLOTHING_ACCESSORIES_STORES_448,RETAIL_ELECTRONICS__APPLIANCE_STORES_443,RETAIL_FOOD__BEVERAGE_STORES_445,RETAIL_FURNITURE__HOME_FURNISHINGS_STORES_442,RETAIL_GASOLINE_STATIONS_447,RETAIL_GENERAL_MERCHANDISE_STORES_452,RETAIL_HEALTH__PERSONAL_CARE_STORES_446,RETAIL_MISCELLANEOUS_STORE_RETAILERS_453,RETAIL_MOTOR_VEHICLE__PARTS_DEALERS_441,RETAIL_NONSTORE_RETAILERS_454,RETAIL_SPORTING_GOODS_HOBBY_MUSIC__BOOK_STORES_451,SPECIAL_EVENT_SALES,TRANSPORTATION__WAREHOUSING_48_49,UNKNOWN_NONCLASSIFIABLE,UTILITIES_22,WHOLESALE_TRADE_DURABLE_GOODS_423,WHOLESALE_TRADE_ELECTRONIC_MARKETS_425,WHOLESALE_TRADE_NONDURABLE_GOODS_424,TOTAL
0,5,American Fork,Cities,002,27413,32519,11.256600,"{'rings': [[[434244.49682960496, 4467294.37416...",360525.0,625281.0,1750000.0,1132810.0,10855679.0,69947.0,1510886.0,25740513.0,368700.0,6514315.0,25000.0,3709354.0,60000.0,12382993.0,2622074.0,4500000.0,4496886.0,90000.0,3964900.0,29535937.0,5219505.0,3463441.0,4360567.0,2896641.0,3000000.0,45182823.0,5071921.0,7479307.0,109956893.0,19033396.0,7849449.0,50000.0,498043.0,36566.0,5750000.0,15336406.0,459562.0,11190788.0,357151108.0
1,12,Beaver City,Cities,002,3137,3104,6.638160,"{'rings': [[[355522.91657235345, 4239879.43388...",988520.0,17144.0,400000.0,7000.0,29553.0,10000.0,457083.0,2263825.0,60000.0,495267.0,1000.0,1308568.0,0.0,1021247.0,142192.0,700000.0,153031.0,0.0,143519.0,1005793.0,102242.0,26765.0,2250000.0,9064.0,3750000.0,554323.0,108995.0,235502.0,828081.0,1871246.0,36255.0,35000.0,20000.0,3747.0,600000.0,413988.0,80000.0,146624.0,20275574.0
2,15,Blanding,Cities,002,3514,3696,13.205500,"{'rings': [[[634403.6869837723, 4163230.893496...",408578.0,45672.0,1000.0,20000.0,305743.0,50000.0,17919.0,600000.0,60000.0,485187.0,5000.0,77542.0,0.0,363436.0,59705.0,900000.0,143740.0,500000.0,59894.0,2720256.0,80220.0,111496.0,2250000.0,3258.0,1500000.0,748656.0,73134.0,124955.0,394683.0,1473315.0,76601.0,0.0,20000.0,527.0,90000.0,308763.0,8000.0,232607.0,14319887.0
3,19,Bountiful,Cities,004,43480,44098,13.209000,"{'rings': [[[426520.60679261835, 4528697.30425...",100000.0,975548.0,4000.0,585659.0,1647021.0,81571.0,1116141.0,11839024.0,935892.0,5678745.0,25000.0,2171121.0,5000.0,5958817.0,1671381.0,5500000.0,3301438.0,900000.0,2053241.0,1898525.0,2654922.0,3819206.0,24807045.0,1691416.0,2500000.0,2278386.0,2195173.0,3755609.0,58937846.0,17497067.0,2248783.0,150000.0,35000.0,16822.0,8500000.0,4763409.0,131656.0,641651.0,183072115.0
4,21,Brigham,Cities,017,18277,19404,24.910801,"{'rings': [[[415000.02685977484, 4598737.23444...",655681.0,186299.0,3000.0,600000.0,1456710.0,28719.0,201424.0,6912003.0,297620.0,2767801.0,15000.0,9242535.0,1000.0,3281464.0,715733.0,3250000.0,796656.0,1000.0,1868664.0,2727400.0,440580.0,1228067.0,11722561.0,507641.0,3407129.0,628164.0,355527.0,896487.0,18008684.0,6288043.0,596162.0,30000.0,45507.0,3732.0,2750000.0,5838242.0,72838.0,243912.0,88071985.0
5,27,Cedar City,Cities,003,30139,33055,36.053398,"{'rings': [[[313136.4164137945, 4175565.793754...",3857794.0,2068165.0,12530.0,306753.0,5459777.0,93952.0,509364.0,18480935.0,406948.0,6172840.0,25000.0,10380705.0,6000.0,4691557.0,1864546.0,5750000.0,2716634.0,450000.0,3114140.0,33835542.0,3632838.0,1340270.0,18613982.0,4286413.0,8010265.0,41470340.0,1985888.0,5

In [10]:
# merge the 2019 sdf with the pivoted data
cities_q2_2019 = cities_sdf.merge(sectors_q2_2019, left_on= "NAME", right_on="City", how="inner")
cities_q2_2019.head(10)

,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,ACCOMMODATION_721,ADMIN__SUPPORT__WASTE_MANAG__REMED_SERVICES_56,AGRICULTURE_FORESTRY_FISHING__HUNTING_11,ARTS_ENTERTAINMENT_AND_RECREATION_71,CONSTRUCTION_23,EDUCATIONAL_SERVICES_61,FINANCE__INSURANCE_52,FOOD_SERVICES__DRINKING_PLACES_722,HEALTH_CARE__SOCIAL_ASSISTANCE_62,INFORMATION_51,MANAGEMENT_OF_COMPANIES__ENTERPRISES_55,MANUFACTURING_31_33,MINING_QUARRYING__OIL__GAS_EXTRACTION_21,OTHER_SERVICES_EXCEPT_PUBLIC_ADMINISTRATION_81,PRIOR_PERIOD_PAYMENTS__REFUNDS,PRIVATE_MOTOR_VEHICLE_SALES,PROFESSIONAL_SCIENTIFIC__TECHNICAL_SERVICES_54,PUBLIC_ADMINISTRATION_92,REAL_ESTATE_RENTAL__LEASING_53,RETAIL_BUILD_MATERIAL_GARDEN_EQUIP__SUPPLIES_DEALERS_444,RETAIL_CLOTHING__CLOTHING_ACCESSORIES_STORES_448,RETAIL_ELECTRONICS__APPLIANCE_STORES_443,RETAIL_FOOD__BEVERAGE_STORES_445,RETAIL_FURNITURE__HOME_FURNISHINGS_STORES_442,RETAIL_GASOLINE_STATIONS_447,RETAIL_GENERAL_MERCHANDISE_STORES_452,RETAIL_HEALTH__PERSONAL_CARE_STORES_446,RETAIL_MISCELLANEOUS_STORE_RETAILERS_453,RETAIL_MOTOR_VEHICLE__PARTS_DEALERS_441,RETAIL_NONSTORE_RETAILERS_454,RETAIL_SPORTING_GOODS_HOBBY_MUSIC__BOOK_STORES_451,SPECIAL_EVENT_SALES,TRANSPORTATION__WAREHOUSING_48_49,UNKNOWN_NONCLASSIFIABLE,UTILITIES_22,WHOLESALE_TRADE_DURABLE_GOODS_423,WHOLESALE_TRADE_ELECTRONIC_MARKETS_425,WHOLESALE_TRADE_NONDURABLE_GOODS_424,TOTAL
0,5,American Fork,Cities,002,27413,32519,11.256600,"{'rings': [[[434244.49682960496, 4467294.37416...",700000.0,566647.0,1250000.0,1600920.0,8794650.0,99011.0,1790289.0,24651818.0,498511.0,9244154.0,20000.0,3599673.0,25000.0,11024408.0,3859810.0,3000000.0,2182874.0,35000.0,3830247.0,23085674.0,7224842.0,7042705.0,4026871.0,3546430.0,2887348.0,41378967.0,5619710.0,6320643.0,89772175.0,7550237.0,7817656.0,20000.0,600000.0,0.0,8250000.0,11443316.0,25000.0,7358148.0,310742734.0
1,12,Beaver City,Cities,002,3137,3104,6.638160,"{'rings': [[[355522.91657235345, 4239879.43388...",1423694.0,31711.0,250000.0,7000.0,35000.0,15000.0,220378.0,2737418.0,50000.0,722352.0,1000.0,698857.0,0.0,889776.0,719732.0,500000.0,96364.0,0.0,100714.0,564664.0,58911.0,23509.0,1750000.0,4000.0,3500000.0,444185.0,72269.0,143038.0,615787.0,587176.0,23446.0,20000.0,4000.0,0.0,600000.0,524356.0,25000.0,203124.0,17662461.0
2,15,Blanding,Cities,002,3514,3696,13.205500,"{'rings': [[[634403.6869837723, 4163230.893496...",1386710.0,48945.0,1000.0,90000.0,210655.0,250000.0,25496.0,700000.0,60000.0,483587.0,5000.0,164496.0,0.0,260314.0,189826.0,800000.0,61873.0,500000.0,93226.0,1948647.0,62024.0,39162.0,2000000.0,7000.0,1250000.0,634468.0,58423.0,125019.0,305191.0,656070.0,43412.0,0.0,25000.0,0.0,100000.0,598802.0,1000.0,402846.0,13588192.0
3,19,Bountiful,Cities,004,43480,44098,13.209000,"{'rings': [[[426520.60679261835, 4528697.30425...",150000.0,2460364.0,2000.0,1105872.0,2074898.0,104656.0,1151100.0,13673812.0,987748.0,5176705.0,20000.0,2685170.0,10000.0,6990239.0,4448948.0,5250000.0,2157862.0,600000.0,2670871.0,1318455.0,2791903.0,3289392.0,20779545.0,1808556.0,2500000.0,1705541.0,1947420.0,3157634.0,53853811.0,7669928.0,1181965.0,90000.0,45000.0,0.0,7000000.0,3651271.0,9000.0,715392.0,165235058.0
4,21,Brigham,Cities,017,18277,19404,24.910801,"{'rings': [[[415000.02685977484, 4598737.23444...",2250000.0,188781.0,3000.0,800000.0,1527976.0,58893.0,292577.0,7047410.0,499834.0,2584387.0,25000.0,6400432.0,1000.0,3013123.0,678568.0,2500000.0,391323.0,1000.0,1731936.0,1916996.0,392600.0,1059660.0,9955541.0,529234.0,3500000.0,1045589.0,485876.0,1491898.0,15865705.0,3295918.0,534664.0,90000.0,70000.0,0.0,2500000.0,5046056.0,10000.0,320490.0,78105467.0
5,27,Cedar City,Cities,003,30139,33055,36.053398,"{'rings': [[[313136.4164137945, 4175565.793754...",6860303.0,1701035.0,4000.0,493035.0,4719307.0,444925.0,621153.0,19225467.0,300552.0,6096848.0,30000.0,7312989.0,25000.0,4677905.0,69081.0,4250000.0,1712156.0,500000.0,3072879.0,25412979.0,4403561.0,1232293.0,15038161.0,3270362.0,6697444.0,35090338.0,1782415.0,3971617.0,18652495.0,590522

In [11]:
# merge the 2019-2020 difference with the sdf
q2_diff = cities_q2_2019[new_names + ['TOTAL']] - cities_q2_2020[new_names + ['TOTAL']]
cities_q2_diff = cities_q2_2019[['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']].merge(q2_diff, left_index=True, right_index=True, how="inner")
cities_q2_diff.head(10)

,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,ACCOMMODATION_721,ADMIN__SUPPORT__WASTE_MANAG__REMED_SERVICES_56,AGRICULTURE_FORESTRY_FISHING__HUNTING_11,ARTS_ENTERTAINMENT_AND_RECREATION_71,CONSTRUCTION_23,EDUCATIONAL_SERVICES_61,FINANCE__INSURANCE_52,FOOD_SERVICES__DRINKING_PLACES_722,HEALTH_CARE__SOCIAL_ASSISTANCE_62,INFORMATION_51,MANAGEMENT_OF_COMPANIES__ENTERPRISES_55,MANUFACTURING_31_33,MINING_QUARRYING__OIL__GAS_EXTRACTION_21,OTHER_SERVICES_EXCEPT_PUBLIC_ADMINISTRATION_81,PRIOR_PERIOD_PAYMENTS__REFUNDS,PRIVATE_MOTOR_VEHICLE_SALES,PROFESSIONAL_SCIENTIFIC__TECHNICAL_SERVICES_54,PUBLIC_ADMINISTRATION_92,REAL_ESTATE_RENTAL__LEASING_53,RETAIL_BUILD_MATERIAL_GARDEN_EQUIP__SUPPLIES_DEALERS_444,RETAIL_CLOTHING__CLOTHING_ACCESSORIES_STORES_448,RETAIL_ELECTRONICS__APPLIANCE_STORES_443,RETAIL_FOOD__BEVERAGE_STORES_445,RETAIL_FURNITURE__HOME_FURNISHINGS_STORES_442,RETAIL_GASOLINE_STATIONS_447,RETAIL_GENERAL_MERCHANDISE_STORES_452,RETAIL_HEALTH__PERSONAL_CARE_STORES_446,RETAIL_MISCELLANEOUS_STORE_RETAILERS_453,RETAIL_MOTOR_VEHICLE__PARTS_DEALERS_441,RETAIL_NONSTORE_RETAILERS_454,RETAIL_SPORTING_GOODS_HOBBY_MUSIC__BOOK_STORES_451,SPECIAL_EVENT_SALES,TRANSPORTATION__WAREHOUSING_48_49,UNKNOWN_NONCLASSIFIABLE,UTILITIES_22,WHOLESALE_TRADE_DURABLE_GOODS_423,WHOLESALE_TRADE_ELECTRONIC_MARKETS_425,WHOLESALE_TRADE_NONDURABLE_GOODS_424,TOTAL
0,5,American Fork,Cities,002,27413,32519,11.256600,"{'rings': [[[434244.49682960496, 4467294.37416...",339475.0,-58634.0,-500000.0,468110.0,-2061029.0,29064.0,279403.0,-1088695.0,129811.0,2729839.0,-5000.0,-109681.0,-35000.0,-1358585.0,1237736.0,-1500000.0,-2314012.0,-55000.0,-134653.0,-6450263.0,2005337.0,3579264.0,-333696.0,649789.0,-112652.0,-3803856.0,547789.0,-1158664.0,-20184718.0,-11483159.0,-31793.0,-30000.0,101957.0,-36566.0,2500000.0,-3893090.0,-434562.0,-3832640.0,-46408374.0
1,12,Beaver City,Cities,002,3137,3104,6.638160,"{'rings': [[[355522.91657235345, 4239879.43388...",435174.0,14567.0,-150000.0,0.0,5447.0,5000.0,-236705.0,473593.0,-10000.0,227085.0,0.0,-609711.0,0.0,-131471.0,577540.0,-200000.0,-56667.0,0.0,-42805.0,-441129.0,-43331.0,-3256.0,-500000.0,-5064.0,-250000.0,-110138.0,-36726.0,-92464.0,-212294.0,-1284070.0,-12809.0,-15000.0,-16000.0,-3747.0,0.0,110368.0,-55000.0,56500.0,-2613113.0
2,15,Blanding,Cities,002,3514,3696,13.205500,"{'rings': [[[634403.6869837723, 4163230.893496...",978132.0,3273.0,0.0,70000.0,-95088.0,200000.0,7577.0,100000.0,0.0,-1600.0,0.0,86954.0,0.0,-103122.0,130121.0,-100000.0,-81867.0,0.0,33332.0,-771609.0,-18196.0,-72334.0,-250000.0,3742.0,-250000.0,-114188.0,-14711.0,64.0,-89492.0,-817245.0,-33189.0,0.0,5000.0,-527.0,10000.0,290039.0,-7000.0,170239.0,-731695.0
3,19,Bountiful,Cities,004,43480,44098,13.209000,"{'rings': [[[426520.60679261835, 4528697.30425...",50000.0,1484816.0,-2000.0,520213.0,427877.0,23085.0,34959.0,1834788.0,51856.0,-502040.0,-5000.0,514049.0,5000.0,1031422.0,2777567.0,-250000.0,-1143576.0,-300000.0,617630.0,-580070.0,136981.0,-529814.0,-4027500.0,117140.0,0.0,-572845.0,-247753.0,-597975.0,-5084035.0,-9827139.0,-1066818.0,-60000.0,10000.0,-16822.0,-1500000.0,-1112138.0,-122656.0,73741.0,-17837057.0
4,21,Brigham,Cities,017,18277,19404,24.910801,"{'rings': [[[415000.02685977484, 4598737.23444...",1594319.0,2482.0,0.0,200000.0,71266.0,30174.0,91153.0,135407.0,202214.0,-183414.0,10000.0,-2842103.0,0.0,-268341.0,-37165.0,-750000.0,-405333.0,0.0,-136728.0,-810404.0,-47980.0,-168407.0,-1767020.0,21593.0,92871.0,417425.0,130349.0,595411.0,-2142979.0,-2992125.0,-61498.0,60000.0,24493.0,-3732.0,-250000.0,-792186.0,-62838.0,76578.0,-9966518.0
5,27,Cedar City,Cities,003,30139,33055,36.053398,"{'rings': [[[313136.4164137945, 4175565.793754...",3002509.0,-367130.0,-8530.0,186282.0,-740470.0,350973.0,111789.0,744532.0,-106396.0,-75992.0,5000.0,-3067716.0,19000.0,-13652.0,-1795465.0,-1500000.0,-1004478.0,50000.0,-41261.0,-8422563.0,770723.0,-107977.0,-3575821.0,-1016051.0,-1312821.0,-6380002.0,-203473.0,-1471677.0,-3723202.0,-7793295.0,-

In [12]:
# Check join output shape
print(cities_q2_2019.shape)
print(cities_q2_2020.shape)
print(cities_q2_diff.shape)

(65, 47)
(65, 47)
(65, 47)


In [13]:
# Confirm join count
l1 = list(q2_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [14]:
# Create file gdb and export sdf to feature class
arcpy.CreateFileGDB_management(".\\Outputs", "sales_tax_revenue.gdb")
cities_q2_2019.spatial.to_featureclass(location=os.path.join(".\\Outputs", "sales_tax_revenue.gdb","utah_q2_2019"))
cities_q2_2020.spatial.to_featureclass(location=os.path.join(".\\Outputs", "sales_tax_revenue.gdb","utah_q2_2020"))
cities_q2_diff.spatial.to_featureclass(location=os.path.join(".\\Outputs", "sales_tax_revenue.gdb","utah_q2_2019_2020_diff"))

'E:\\Projects\\Sales_Tax\\Outputs\\sales_tax_revenue.gdb\\utah_q2_2019_2020_diff'

### Analysis

In [15]:
# 10 cities with greatest loss of revenue
cities_q2_diff[['NAME', 'POPLASTEST', 'AREA_SQMI', 'TOTAL']].sort_values('TOTAL').head(10)

,NAME,POPLASTEST,AREA_SQMI,TOTAL
62,West Jordan,116046,32.306999,-91207296.0
54,St George,87178,77.209900,-84029566.0
51,Spanish Fork,39961,16.249701,-57141263.0
57,Tooele City,35251,24.087099,-48598585.0
63,West Valley City,136401,35.799702,-46907870.0
0,American Fork,32519,11.256600,-46408374.0
20,Layton,77303,22.576200,-42043505.0
5,Cedar City,33055,36.053398,-40919339.0
23,Logan,51619,18.485800,-40702065.0
34,Orem,97521,18.628000,-40443599.0


In [16]:
# 10 cities with least loss of revenue
cities_q2_diff[['NAME', 'POPLASTEST', 'AREA_SQMI', 'TOTAL']].sort_values('TOTAL', ascending=False).head(10)

,NAME,POPLASTEST,AREA_SQMI,TOTAL
45,Salt Lake City,200591,110.734001,217551883.0
27,Moab,5322,4.802460,43345377.0
35,Park City,8504,20.341700,42810440.0
30,Murray,49308,12.293700,40848514.0
48,South Jordan,74149,22.263201,25277957.0
52,Springdale,609,4.594880,21011489.0
12,Farmington,24514,10.035100,17939769.0
43,Roosevelt,7070,6.757070,9635486.0
17,Kanab,4798,14.473200,6206268.0
59,Vernal,10370,4.632710,3023499.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)

In [17]:
# cities_q2_2019[cities_q2_2019['NAME'] == "Salt Lake City"]
#cities_q2_2020[cities_q2_2020['NAME'] == "Salt Lake City"]